<a href="https://colab.research.google.com/github/tusker4/Sesac_Saltlux_DeepLearning/blob/main/7_%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_MNIST_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 덤프

# 목적

- keras 대비 코드량/난이도 체크
- 객체 지향 스타일 체크
- torch 스타일 체크

# 모듈 가져오기

In [ ]:
import torch
# 인공 신경망의 레이어들
import torch.nn as nn
# 망구축용 함수모음
import torch.nn.functional as F
# 최적화 도구
import torch.optim as optim

# 데이터 공급자 => 학습에 필요한 데이터 제공
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [ ]:
# GPU
DEVICE = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )
DEVICE

device(type='cpu')

# 데이터 준비

- 데이터 공급자를 세팅, 훈련시 공급하게 준비

In [ ]:
# 배치 사이즈 (미니 배치 학습, 에포크, 반복회수:이터레이터, 1회학습시 데이터량)
BATCH_SIZE = 64 # 설정

In [ ]:
# 훈련 데이터 공급자
train_loader = DataLoader(
    # 데이터 설정(커스텀, 사전에 준비된 내용)
    datasets.FashionMNIST(
        root='./data',      # 데이터가 저장된 위치
        train=True,         # 훈련용 데이터
        download=True,      # 데이터가 없다면 다운로드
        # 이미지 전처리
        transform=transforms.Compose([
            transforms.ToTensor(),    # 데이터를 텐서 형식으로 공급 받겠다
            transforms.Normalize( 0.2, 0.3 ) # 평균, 표준편차를 설정해서 정규화 처리 수행
        ])
    ),                      # 공급할 데이터
    batch_size=BATCH_SIZE,  # 배치사이즈
    shuffle=True            # 데이터 셔플
)

100%|██████████| 26421880/26421880 [00:01<00:00, 16973430.83it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 274046.18it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4966107.06it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 22704812.82it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
# 테스트 데이터 공급자
test_loader = DataLoader(
    # 데이터 설정(커스텀, 사전에 준비된 내용)
    datasets.FashionMNIST(
        root='./data',      # 데이터가 저장된 위치
        train=False,         # 훈련용 데이터
        download=True,      # 데이터가 없다면 다운로드
        # 이미지 전처리
        transform=transforms.Compose([
            transforms.ToTensor(),    # 데이터를 텐서 형식으로 공급 받겠다
            transforms.Normalize( 0.2, 0.3 ) # 평균, 표준편차를 설정해서 정규화 처리 수행
        ])
    ),                      # 공급할 데이터
    batch_size=BATCH_SIZE,  # 배치사이즈
    shuffle=True            # 데이터 셔플
)

- 동일한 소스에서 훈련, 테스트 데이터를 공급하므로, 중복 데이터 가능성 열어두고 진행

# 인공신경망 구축 - 객체 지향 스타일



```
- 입력층
- 은닉층
    L 합성곱층
    L 풀링층
    L 합성곱층
    L 풀링층
    L flattern
    L 전결합층
- 출력층
```



In [ ]:
class Net(nn.Module):
    #name = 'torch cnn'
    # 생성자
    def __init__(self):
        '''
            1. 명시적 상속 => 부모 클레스(super)의 생성자 호출 진행
            2. 각 신경망의 레이어들 생성(맴버 변수로)
        '''
        super(Net, self).__init__()
        # 맴버 접근은 객체명.맴버
        # FashionMNIST -> 28x28 grayscale(1채널)
        # 풀링, 플랫툰 등은 신경망 연결시 세팅(만들기 나름)
        self.conv1   = nn.Conv2d(1, 32, 5, 1, padding='same')    # 합성곱 1층
        self.conv2   = nn.Conv2d(32,32*2, 5, 1, padding='same')  # 합성곱 2층
        self.dropout = nn.Dropout2d(0.1) # 과적합 방지
        # 28 -> 14 -> 7 -> flattern : 7*7*32*2 (4D->2D)
        self.fc      = nn.Linear(7*7*32*2, 1024) # 3136 -> 1024로 수렴
        self.output  = nn.Linear(1024, 10)
        pass

    # 맴버함수, 부모의 함수를 재정의(overide)
    # 순전파 신경망 (x->y), 인공신경망 연결
    # 역전파 신경망을 재정의 하지 않으면 부모가 세팅한 방식 그대로 사용한다는 의미
    # 순전파 방향으로 구성된 인공신경망(시퀀스 형태)를 리턴
    def forward(self, x):  # x는 입력층
        # 합성곱 1f (합성곱 + 최대풀링(레이어로 않하고 함수로 처리, 설정)포함)
        # 이미지 크기(h,w) : 28 -> 14
        # 객체를 함수처럼 사용 __call__ 구현되어 있으면 전달 가능
        x = F.relu( F.max_pool2d( self.conv1(x), 2 ) ) # 2:커널사이즈, stride는 동일값
        # 합성곱 2f conv2에 dropout 추가
        # 이미지 크기(h,w) : 14 -> 7
        x = F.relu( F.max_pool2d( self.dropout(self.conv2(x)), 2 ) )
        # 4D -> 2D : Flattern => torch에서는 view => 데이터의 순서가 바뀌지 않는다
        # -1은 뒤를 채우고 나머지 수치는 앞을 채워라.
        # 훈련시, 데이터를 가변적으로 넣을수 있다
        x = x.view( -1, 7*7*32*2)
        # 3131 -> 1024
        x = self.dropout(F.relu( self.fc(x) ))
        # 1023 -> 10
        # softmax 혹은 log_softmax 사용 -> 데이터가 원본이면(softmax)
        # 정규화를 통해서 분포가 변경되고, 데이터도 변경됨 -> log_softmax 진행
        return F.log_softmax( self.output( x ), dim=1 )  # dim=1 출력값 shape 조정



        pass
    pass

# 인공 신경망 생성후 cpu 혹은 cuda 사용 지정
model = Net().to(DEVICE)
model

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (dropout): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=3136, out_features=1024, bias=True)
  (output): Linear(in_features=1024, out_features=10, bias=True)
)

# 학습에 필요한 도구 준비

- 케라스에서는 모델 컴파일
    - 최적화도구(GD, SGD,.)
    - 손실함수(크로스엔트로피(이진:로지스틱회귀+시그모이드,다중분류:소프트맥스회귀))
    - 평가지표(정확도,...)

- 토치
    - 학습 및 테스트 진행시 손실함수, 평가지표 사용 -> 학습 함수, 테스트 함수를 만들어서 내부에서 체크

In [ ]:
# 최적화 도구
# model.parameters() : 신경망 구성으로 발생된 w와 b를 접급할수 있는 객체 => 최적화의 대상
# lr : 학습률
# momentum : 가속도
optimzer = optim.SGD( model.parameters(), lr=0.01, momentum=0.5)

# 훈련

## 훈련용 함수

In [ ]:
def train( model, train_loader, optimzer, epoch ):
    '''
        - model : 인공신경망, 학습할 모델
        - train_loader : 훈련 데이터 공급자
        - optimzer : 최적화 도구
        - epoch : 에폭 정보 (1 ~ 10)
    '''
    # 1. 모델을 학습 모드로 전환
    model.train()
    # 2. 미니 배치 학습 진행 -> 전체 데이터를 배치사이즈 대비 나눈값양으로 훈련진행
    for idx, (data, target) in enumerate( train_loader ): # 인덱스, 피처데이터, 정답데이터
        # 3. cpu, gpu를 데이터에 설정
        data   = data.to(DEVICE)
        target = target.to(DEVICE)
        # 4. 최적화 도구 초기화, 최적화한 내용은 밑에서 기록되고, 새로 학습 진행되면 초기화
        #    한번의 학습이 완료되면 (Iteration이 한번 끝나면) gradients가 기록(밑에서)되고
        #    다음 학습을 위해서 항상 0으로 만들어 줘야함
        optimzer.zero_grad()
        # 5. 모델에 데이터를 주입하여 결과를 얻는다
        ouptut = model( data ) # 예측
        # 6. 실제값과 예측값 사이의 차이 계산 => 손실함수
        loss   = F.cross_entropy( ouptut, target )
        # 7. 오차역전파 진행 : y -> x 이동 => 가중치 업데이트 => 최적화 수행
        #    gradients가 기록한다
        loss.backward()
        # 8. 최종적으로 만들어진 최적 파라미터(w, b)를  모델에 최종 반영
        optimzer.step()
        # 9. 로그 출력 : vervose에서 컨트롤
        if idx % 200 == 0: # 특정단위가 도래하면 로그 출력
            print( f'Epoch:{epoch}\t batch cnt:{idx}\t loss={loss.item()}')
        pass
    # 체크 포인트 - 모델 덤프 (pt or pth)
    torch.save({
        'model' : 'train-model'
    }, f'./model-checkpoint-{epoch}.pt')
    pass

## 테스트용 함수

In [ ]:
def test( model, test_loader ):
    '''
        세대별(1epoch, 2epoch,..) 학습 결과를 모아서 평균 손실, 정확도 출력
        - model : 인공신경망, 학습할 모델
        - test_loader : 테스트 데이터 공급자
    '''
    # 0. 손실, 정확도를 담을 값을 누적해서 데이터 총개수로 나누면 체크 완료
    loss, acc = (0,0)
    # 1. 모델을 테스트 모드로 전환
    model.eval()
    # 2. 테스트가 진행되는 동안 모든 결과는 기록하지 않는다 -> 학습에 영향을 않미친다
    with torch.no_grad(): # 계산 X
        # 3. 미니 배치 학습 진행 -> 전체 데이터를 배치사이즈 대비 나눈값양으로 훈련진행
        for idx, (data, target) in enumerate( train_loader ): # 인덱스, 피처데이터, 정답데이터
            # 4. cpu, gpu를 데이터에 설정
            data, target   = data.to(DEVICE), target.to(DEVICE)
            # 5. 모델에 데이터를 주입하여 결과를 얻는다
            ouptut = model( data ) # 예측
            # 6. 실제값과 예측값 사이의 차이 계산 => 손실함수
            #    64개 데이터를 가져와서 손실계산 => 64개의 손실값이 등장
            #    reduction='sum' : 텐서 맴버들을 모두 더해라
            #    item() 텐서의 값 추출(맴버가 1개만 가능)
            loss   += F.cross_entropy( ouptut, target, reduction='sum').item()
            # 7. 정확도 -> 예측값 필요
            #    10개로 분류된 결과는 확률 -> 가장 높은 확률 -> 분류값
            #    numpy.argmax() : 최고값을 가진 인덱스 번호 리턴
            pred = ouptut.max(1, keepdim=True)[1] # 최고값을 가진 인덱스 번호를 shape유지해서 리턴
            #    target에 정답 인덱스가 있다
            # 8. 정확도 체크
            #    target과 pred간 데이터별 예측값이 일치 => 인덱스 값이 일치 체크
            #    행렬비교 => True, False,... => 1, 0,.. => sum() => 텐서, 값1개 => item() => 값
            #    64개를 예측해서 55개 맞췄다
            acc  += pred.eq( target.view_as( pred ) ).sum().item() # 맞춘개수

            pass
    # 특정 epoch의 평균 손실, 평균 정확도
    mean_loss = loss / len( test_loader.dataset )
    mean_acc  = acc  / len( test_loader.dataset ) * 100 # 퍼센트
    return mean_loss, mean_acc
    pass

## 훈련 진행

In [ ]:
EPOCHS = 10 # 총 10세대 학습 진행
for epoch in range(1, EPOCHS+1):
    # 훈련
    train( model, train_loader, optimzer, epoch )
    # 테스트
    mean_loss, mean_acc = (  model, test_loader )
    # 로그출력
    print( f'epoch:{epoch}\t loss={mean_loss}\t acc={mean_acc}')

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch:1	 batch cnt:0	 loss=2.3151438236236572
Epoch:1	 batch cnt:200	 loss=0.7033793926239014
Epoch:1	 batch cnt:400	 loss=0.6762149333953857
Epoch:1	 batch cnt:600	 loss=0.44463682174682617
Epoch:1	 batch cnt:800	 loss=0.3419739305973053
epoch:1	 loss=Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (dropout): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=3136, out_features=1024, bias=True)
  (output): Linear(in_features=1024, out_features=10, bias=True)
)	 acc=<torch.utils.data.dataloader.DataLoader object at 0x7f2ec4b2fd60>
Epoch:2	 batch cnt:0	 loss=0.385941743850708
Epoch:2	 batch cnt:200	 loss=0.33146223425865173
Epoch:2	 batch cnt:400	 loss=0.4148309528827667
Epoch:2	 batch cnt:600	 loss=0.37486666440963745
Epoch:2	 batch cnt:800	 loss=0.2281140238046646
epoch:2	 loss=Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (conv2): Conv

# 개별 모듈 테스트

In [ ]:
a = torch.Tensor( [1,1,2,0] )
b = torch.Tensor( [0,0,1,1] )

F.cross_entropy( a, b, reduction='sum',  ).item()

3.253046989440918

In [ ]:
a = torch.Tensor( [1,1,2,0] )
b = torch.Tensor( [[1,0,1,1]] )
a.size(), b.size()

(torch.Size([4]), torch.Size([1, 4]))

In [ ]:
# 특정 차원의 텐서를 특정 차원의 텐서 형태로 변환, 비교, 합산, 값추출
a.view_as(b), b.eq(a.view_as(b)), b.eq(a.view_as(b)).sum(), b.eq(a.view_as(b)).sum().item()

(tensor([[1., 1., 2., 0.]]),
 tensor([[ True, False, False, False]]),
 tensor(1),
 1)

In [ ]:
tensor = torch.arange(16).view(4,4)
print( tensor, tensor.size() )
# 4개의 데이터에서 각각 최고값을 가진 값 혹은 인덱스 추출
print( '-'*10)
print( tensor.max() ) # 전체 구성원중 가장 높은값 추출
print( '-'*10)
print( tensor.max(1) ) # 1 => dim 값이 두번째 => 2차원
print( '-'*10)
print( tensor.max(1)[1] ) # 각 데이터에서 인덱스 번호 3번이 최대값의 자리이다
print( '-'*10)
print( tensor.max(1, keepdim=True)[1] ) # 차원유지 => 2차원유지, 최대값을 가진 인덱스만 추출

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]]) torch.Size([4, 4])
----------
tensor(15)
----------
torch.return_types.max(
values=tensor([ 3,  7, 11, 15]),
indices=tensor([3, 3, 3, 3]))
----------
tensor([3, 3, 3, 3])
----------
tensor([[3],
        [3],
        [3],
        [3]])


# 모델 덤프

## 모델 전체 저장, 불러오기

## 모델 전체 저장, 불러오기

# 모델 덤프